Def matrizes_global(): Esse método realiza a montagem das matrizes globais de rigidez e de massa, e a adição de engastes nos nós especificados. Tem como entrada o self associado a classe Estrutura, então é feito um loop que percorre todos os elementos que compõem a estrutura. 
Para cada um desses elementos, é chamado o método element a fim de calcular as matrizes elementares de massa e rigidez. Em seguida é definidos os graus de liberdades(dofs) considerados.Cada nó possui 6 dofs (x,y,z,rx,ry,rz), então para um elemento de 2 nó os dofs são: x1,y1,z1,rx1,ry1,rz1,x2,y2,z2,rx2,ry2,rz2. Para fazer a montagem correta da matriz global, os dofs são determinados em função dos nós do elemento, descrito na forma de uma sequência dada por: número de dofs * nó + i , com i variando de 0 até 5. Em seguida são montados dois loops aninhando para fazer a adição de cada entrada presente na matriz elementar na sua respectiva posição na matriz global. 
Após isso é feita a aplicação dos engastes, anteriormente o engaste estava sendo considerado como um aumento na rigidez em alguns pontos especificos na matriz. Agora o engaste foi atualizado para ter um metodo proprio: aplicar_engastes. Esse metodo recebe como entradas o self associado a classe Estrutura, os nós que serão engastados e os dofs que serão restringidos.

In [ ]:
def aplicar_engastes(self, nodes, dofs):
        """
        Aplica engastes nos nós especificados, restringindo os graus de liberdade.
        
        Args:
        nodes: Lista dos nós nos quais os engastes serão aplicados.
        dofs: Lista dos graus de liberdade que serão restringidos para cada nó.
        """
        for node in nodes:                                          #Laço para selecionar cada nó que será engastado
            for dof in dofs:                                        #Laço para selecionar quais graus de liberdade serão fixados
                index = node * self.num_dofs_per_node + dof         #Identificação da entrada da matriz que precisa ser restringida pelo engaste        
                self.K_global[index, index] = 10**10                # Um valor suficientemente grande para simular um engaste 

In [1]:
 def matrizes_global(self):
        """
        Monta as matrizes globais de rigidez e massa da estrutura,
        e aplica engastes nos nós especificados.
        """
        #Calculando as matrizes de rigidez e massa de cada elemento
        for element in self.elements:
            node1,node2=element
            k_e,m_e= self.element(element)
            #Montando as matrizes globais
            #           x1      y1          z1        rx1       ry1         rz1       x2        y2          z2        rx2        ry2        rz2        
            dofs = [6*node1, 6*node1+1, 6*node1+2, 6*node1+3, 6*node1+4, 6*node1+5, 6*node2, 6*node2+1, 6*node2+2, 6*node2+3, 6*node2+4, 6*node2+5]
            for i in range(len(dofs)):
                for j in range(len(dofs)):
                    self.K_global[dofs[i], dofs[j]] += k_e[i, j]
                    self.M_global[dofs[i], dofs[j]] += m_e[i, j]              

        self.aplicar_engastes([0, 2, 4, 5], [0, 1, 2])

        # Converte a matriz para DataFram
        df_1 = pd.DataFrame(self.K_global)
        df_2 = pd.DataFrame(self.M_global)
        # Salva o arquivo DataFrame como csv
        df_1.to_csv('Matriz_Global_Rigidez.csv', index=True, header=True)
        df_2.to_csv('Matriz_Global_Massa.csv', index=True, header=True)
        return self.K_global,self.M_global

Após a aplicação dos engastes é utilizada a bibilioteca pandas para a exportação das matrizes globais em formato csv. Tal aplicação ocorre em decorrencia do tamanho que as matrizes podem alcançar conforme forem adicionados mais elementos a estrutura, impossibilitando de compreender os resultados dela apenas printando a matriz. Ao salva-las como arquivo csv é possivel abri-las em qualquer documento de planilhas como o google sheets ou o excel, garantindo melhor compreensão.
Por fim a def matrizes_global retorna a matriz self.K_global,self.M_global

Para verificar a robustez desse codigo foram realizados alguns testes com os seguintes objetivos: verificar se a montagem da matriz global estava correta, verificar se a montagem funcionava com mais elementos e coordenadas.
Para os testes foram tomadas as seguintes ações:
1. Executar o codigo com as coordenadas e elementos do chassi simplificado,chassi completo, com apenas 2 elementos do chassi simplificado que não tivessem conectados.
2. Exportar as matrizes resultantes desses codigos para o google sheets a fim de conseguir analisar mais facilmente.
Resultados:
A matriz proveniente de apenas 2 elementos estava condizente com a matriz elemental, indicando que a montagem da global estava correta, e as posições também estavam de acordo com o esperado.
Ao comparar essa matriz com a matriz do chassi simplificado é possivel enxergar como a rigidez e a massa se alterou com a adição de mais elementos.
No caso do chassi completo, é perceptivel que as matrizes estão com a dimensão correta e, partindo do fato que a montagem nos outros casos estão corretas, o resultado encontrado está correto.
Em decorrencia do tamanho das matrizes é dificil disponibiliza-las aqui, portanto o acesso às planilhas do excel contendo elas está disponibilizado abaixo:

https://docs.google.com/spreadsheets/d/1IbtaBDqhuXgIPSQSf_1EzJ7L3sCnE_NnsYZpywnPHV0/edit?usp=sharing  Para as matrizes de rigidez

https://docs.google.com/spreadsheets/d/1k-Xl51AwuAjeHDZYEZA6NkSaxys0x3svV4zk6TYjZWA/edit?usp=sharing  Para as matrizes de massa
